In [ ]:
# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#Resolvendo problema do chromeDrive
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
# Pandas
import pandas as pd
# RegEx / Regular Expressions
import re

In [ ]:
case = 'https://loja.caseih.com.br/search?text='
nh = 'https://loja.newhollandag.com.br/search?text='
rech = 'https://rech.com/busca?q='

In [ ]:
def contains_number(string):
    return any(char.isdigit() for char in string)

In [ ]:
def findOEM(text):
    print(text)
    oem = []
    if 'VV' in text:
        indice = text.find('VV')
        text = text.replace(text[indice:], '')
    if 'MCL' in text:
        cod_mcl = text.find('MCL')
        copy_text = text
        text = text[:cod_mcl]
        cod_mcl = copy_text[cod_mcl:]
        cod_mcl = cod_mcl.replace(' ', '')
        if contains_number(cod_mcl):
            if len(cod_mcl) > 5:
                oem.append(cod_mcl)
    if not contains_number(text):
        return oem
    num = re.search(r'\d+', text)
    indice = text.find(num.group())
    lastspace = text.rfind(' ', 0, indice)
    remover = [text[:lastspace], '.', '-']
    for rm in remover:
        text = text.replace(rm, '')
    text = text.replace(' ', '/')
    text = text.split('/')
    for txt in text:
        if contains_number(txt):
            if len(txt) > 5:
                if not contains_number(txt[-1]):
                    txt = txt.replace(txt[-1], '')
                oem.append(txt)
    return oem

In [ ]:
def nh_case(nav, cod, index):
    try:
        btn = nav.find_element(By.CLASS_NAME, 'product-card')
        link = btn.find_element(By.CLASS_NAME, 'stretched-link').get_attribute("href")
        nav.get(link)
        nome = nav.find_element(By.CLASS_NAME, 'product-title__product-name').text
#         nome = re.sub('[0-9]', '', nome)
        codigo = nav.find_element(By.CLASS_NAME, 'product-code').text
        elimg = nav.find_elements(By.CLASS_NAME, 'owl-carousel__images')
        src = []
        for img in elimg:
            img = img.get_attribute('src')
            if 'loja' not in img:
                src.append(img)
        print(len(src))
        src = list(dict.fromkeys(src))
        keys = []
        values = []
        for k in nav.find_elements(By.CLASS_NAME, 'product-specifications-item__title'):
            keys.append(k.text)
        for v in nav.find_elements(By.CLASS_NAME, 'product-specifications-item__value'):
            values.append(v.text)
        dict_infos = dict(zip(keys, values))
        
        print(nome)
        print(codigo)
        print(len(src))
        for img in src:
            print(img)
        print(dict_infos)
    except:
        pass
def nhCase(nav, cod, index):
    try:
        nav.get(str(nh + cod))
        nh_case(nav, cod, index)       
        nav.get(str(case + cod))
        nh_case(nav, cod, index)
    except:
        pass

In [ ]:
def rechSearch(nav, cod, index):
    elemento = WebDriverWait(nav, 10).until(EC.presence_of_element_located((By.ID, 'search-input')))
    elemento.send_keys(str(cod))
    elemento.send_keys(Keys.ENTER)
    try:
        elemento = nav.find_element(By.ID, 'showcase')
        elementos = elemento.find_elements(By.CLASS_NAME, 'y-item-wrapper-showcase')
        urls = []
        for elemento in elementos:
            url = elemento.find_element(By.TAG_NAME, 'link').get_attribute('href')
            urls.append(url)
            print(url)
        for url in urls:
            src = []
            keys = []
            values = []
            nav.get(str(url))
            nav.implicitly_wait(0.8)
            elementos = nav.find_elements(By.CLASS_NAME, 'zoomImg')
            for elemento in elementos:
                src.append(elemento.get_attribute('src'))
            nome = nav.find_element(By.CLASS_NAME, 'product__name').text
            codigo = nav.find_element(By.CLASS_NAME, 'product__partcode').text
            descricao = nav.find_element(By.CLASS_NAME, 'panel-description').text
            elemento = nav.find_element(By.CLASS_NAME, 'table-specifications')
            elementos = elemento.find_elements(By.CSS_SELECTOR, '.product .product-details table tr th')
            for k in elementos:
                keys.append(k.text)
            elementos = elemento.find_elements(By.CSS_SELECTOR, '.product .product-details table tr td')
            for v in elementos:
                values.append(v.text)
            dict_infos = dict(zip(keys, values))
            for x in dict_infos.keys():
                y = dict_infos.get(x)
                y = y.replace('\n', ', ')
                dict_infos.update({x : y})
            print(nome)
            print(codigo)
            print(descricao)
            print(dict_infos)
            
    except:
        pass

In [ ]:
def loginRech(nav):
    nav.get('https://www.rech.com/login?targetUri=%2F')
    elemento = nav.find_element(By.ID, 'username')
    elemento.send_keys('yuri@cotripal.com.br')
    elemento = nav.find_element(By.ID, 'password')
    elemento.send_keys('Testes1!')
    elemento = nav.find_element(By.ID, 'submit')
    elemento.click()
    nav.implicitly_wait(0.8)
    rechSearch(nav, 87041572, 1)

In [ ]:
chrome_options = webdriver.ChromeOptions()
servico = Service(ChromeDriverManager().install())
#chrome_options.add_argument('headless')
nav = webdriver.Chrome(options=chrome_options, service=servico)
# produtos = dict(zip(query['ID'], query['Produto']))

# nhCase(nav, '44906471', '1')
# sulmatreSearch(nav, '87617623', '1')
# loginRech(nav)
commersulSearch(nav, 756397, 1)

In [ ]:
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('headless')
nav = webdriver.Chrome(options=chrome_options)
nav.get('https://mcl.ind.br/produtos')